In [2]:
!pip install learn2learn --quiet
!pip install import_ipynb --quiet

In [3]:
import import_ipynb
import utils
import models
utils.hide_toggle('Imports 1')

ModuleNotFoundError: ignored

In [66]:
from IPython import display
import torch
from sklearn.manifold import TSNE
from matplotlib import pyplot as plt
from l2lutils import KShotLoader
from IPython import display
utils.hide_toggle('Imports 2')

ModuleNotFoundError: ignored

In [6]:
#I added this
from google.colab import drive
drive.mount('/content/drive')
# %ls drive/MyDrive/'Colab Notebooks'/'Meta_Learning_hw2'
!cp drive/MyDrive/'Colab Notebooks'/'Meta_Learning_hw2'/models.ipynb .
!cp drive/MyDrive/'Colab Notebooks'/'Meta_Learning_hw2'/utils.ipynb .


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import learn2learn as l2l
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import models # I added

In [75]:
class MAN(nn.Module):
    def __init__(self,dims=[20,32,32],n_classes=2,lr=1e-3):
        super(MAN,self).__init__()
        self.n_classes = n_classes
        self.mlp = models.MLP(dims=dims,task='embedding')
        self.attn = nn.Softmax(dim=1)
        self.optimizer = optim.Adam(self.parameters(),lr=lr)

    def forward(self,X,d_train):
        # INSERT YOUR CODE HERE
        train_embed =self.mlp(d_train[0]) #train_embed.shape=[25,32]=[shots*ways,Last output size of network_f] #ntworf_f=network_g in this assignment
        test_embed  =self.mlp(X)          #test_embed.shape=[25,32]=[shots*ways,Last output size of network_g] #shots=ways=5 for train and test dataset of meta_train dataset 
        similarities=self.cos(test_embed,train_embed)
        softmax     =nn.Softmax(dim=1) 
        
        similarities_softmax=softmax(similarities) # softmax of the similarities returned by cos method
        # print(similarities_softmax.sum(dim=1))

        unique_labels=d_train[1].unique()
        # print(unique_labels)
        preds        =torch.empty(X.shape[0],len(unique_labels))
        
        
        for i,label in enumerate(unique_labels):
          preds[:,i]=similarities_softmax[:,d_train[1]==label].sum(dim=1)
        # print(preds.shape)
        return preds

    def cos(self,target,ss):

      '''
      example= D_train and D_test of D_meta_train is 5-shot-5-ways 
      target.shape= (batch,embedding_dim) , here batch=5*5=25, ss_size=5*5=25 
      '''
        # compute cosine similarities between 
        # target (batch,embedding_dim) and support set ss (ss_size,embedding_dim)
        # return (batch,ss_size)
        # INSERT YOUR CODE HERE

      cos_sim=nn.CosineSimilarity(dim=1, eps=1e-8)
      similarities=torch.zeros(target.shape[0],ss.shape[0])
      # print("target shape:", target.shape)
      # print("ss shape:", ss.shape)
      for i in range(target.shape[0]):
        # print(target[i].shape)
        # print(target[i].repeat(ss.shape[0],1).shape)
        # print(ss.shape)
        similarities[i]=cos_sim(target[i].repeat(ss.shape[0],1),ss)  # each row contains similarity value of one x_test data  with the train_data
        # print(similarities[i].shape)
        # print(similarities)

        # break
      return similarities



In [74]:
# import torch
# import numpy as np
# man=MAN(dims=[20,32,32])
# X_test=torch.randn(10,20)
# X_train=torch.randn(25,20)
# y_train=torch.Tensor(np.random.randint(5, size=25)).int()
# d_train=(X_train,y_train)
# man(X_test,d_train)

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000], grad_fn=<SumBackward1>)


torch.Size([2, 3])
torch.Size([2, 3])


In [ ]:
# import torch
# import numpy as np

# ss = torch.randn(25, 32)
# target= torch.randn(25, 32)
# cos_sim=nn.CosineSimilarity(dim=1, eps=1e-8)
# similarities=torch.empty(target.shape[0],ss.shape[0])
# for i,t in enumerate(target):
#   similarities[i]=cos_sim(t.repeat(ss.shape[0],1),ss)
# print(similarities.shape)
# cos_sim(target[0].repeat(ss.shape[0],1),ss).shape
# # torch.stack((input1,input2),1).shape

torch.Size([25])

In [48]:
# m = nn.Softmax(dim=1)
# input = torch.randn(2, 3)
# output = m(input)
# print(output.sum(1))

tensor([1., 1.])
